### Preliminares


In [ ]:
from numpy import ndarray
from typing import Dict, List, Tuple
import matplotlib.pyplot as plt
from IPython import display
#plt.style.use('seaborn-white')
%matplotlib inline

from copy import deepcopy
from collections import deque
from scipy.special import logsumexp

#### Definimos las funciones de activación a utilizar

In [ ]:
def sigmoid(x: ndarray):
    return 1 / (1 + np.exp(-x))


def dsigmoid(x: ndarray):
    return sigmoid(x) * (1 - sigmoid(x))


def tanh(x: ndarray):
    return np.tanh(x)


def dtanh(x: ndarray):
    return 1 - np.tanh(x) * np.tanh(x)


def softmax(x, axis=None):
    return np.exp(x - logsumexp(x, axis=axis, keepdims=True))


def batch_softmax(input_array: ndarray):
    out = []
    for row in input_array:
        out.append(softmax(row, axis=1))
    return np.stack(out)
    

##### Optimizador de redes neuronales recurrentes

In [ ]:
class RNNOptimizer(object):
    # Constructor de la clase optimizador para redes neuronales recurrentes.
    # Args:
    #   lr (float): Tasa de aprendizaje, con un valor por defecto de 0.01.
    #   gradient_clipping (bool): Indica si se aplica clipping a los gradientes,
    #                             activado por defecto.
    def __init__(self, lr: float = 0.01, gradient_clipping: bool = True) -> None:
        self.lr = lr  # Almacena la tasa de aprendizaje.
        self.gradient_clipping = gradient_clipping  # Almacena la configuración de clipping.
        self.first = True  # Variable auxiliar, posiblemente para controlar la primera actualización.

    # Método que ejecuta un paso de optimización sobre todos los parámetros del modelo.
    def step(self) -> None:
        # Itera sobre cada capa del modelo.
        for layer in self.model.layers:
            # Itera sobre cada parámetro de la capa.
            for key in layer.params.keys():
                # Si el clipping de gradientes está activado, aplica clipping.
                if self.gradient_clipping:
                    # Los gradientes se limitan a estar entre -2 y 2 para evitar la explosión de gradientes.
                    np.clip(layer.params[key]['deriv'], -2, 2, layer.params[key]['deriv'])

                # Llama a la regla de actualización para ajustar el valor del parámetro
                # usando la tasa de aprendizaje y el gradiente actual.
                self._update_rule(param=layer.params[key]['value'],
                                  grad=layer.params[key]['deriv'])

    # Método abstracto para definir la regla de actualización de parámetros.
    # Debe ser implementado por subclases.
    def _update_rule(self, **kwargs) -> None:
        raise NotImplementedError("Este método debe ser implementado por subclases.")


#### Algunos optimizadores

In [ ]:
class SGD(RNNOptimizer):
    # Constructor de la clase SGD (Stochastic Gradient Descent).
    # Args:
    #   lr (float): Tasa de aprendizaje.
    #   gradient_clipping (bool): Indica si se activa el clipping de gradientes.
    def __init__(self, lr: float = 0.01, gradient_clipping: bool = True) -> None:
        super().__init__(lr, gradient_clipping)  # Inicializa la clase base

    # Método específico de SGD para actualizar los parámetros según el gradiente.
    def _update_rule(self, **kwargs) -> None:
        update = self.lr * kwargs['grad']  # Calcula la actualización del parámetro.
        kwargs['param'] -= update  # Actualiza el parámetro restando la actualización.
        
class AdaGrad(RNNOptimizer):
    # Constructor de la clase AdaGrad.
    # Args:
    #   lr (float): Tasa de aprendizaje.
    #   gradient_clipping (bool): Indica si se activa el clipping de gradientes.
    def __init__(self, lr: float = 0.01, gradient_clipping: bool = True) -> None:
        super().__init__(lr, gradient_clipping)  # Inicializa la clase base
        self.eps = 1e-7  # Un pequeño número para evitar división por cero en la actualización.

    # Método que ejecuta un paso de optimización sobre todos los parámetros del modelo.
    def step(self) -> None:
        if self.first:
            self.sum_squares = {}  # Diccionario para almacenar la suma acumulada de los cuadrados de los gradientes.
            # Inicializa sum_squares para cada parámetro en cada capa.
            for i, layer in enumerate(self.model.layers):
                self.sum_squares[i] = {}
                for key in layer.params.keys():
                    self.sum_squares[i][key] = np.zeros_like(layer.params[key]['value'])
            self.first = False  # Marca la inicialización como completa.

        for i, layer in enumerate(self.model.layers):
            for key in layer.params.keys():
                if self.gradient_clipping:
                    # Si el clipping de gradientes está activado, aplica clipping.
                    np.clip(layer.params[key]['deriv'], -2, 2, layer.params[key]['deriv'])

                # Actualiza cada parámetro utilizando la regla específica de AdaGrad.
                self._update_rule(param=layer.params[key]['value'],
                                  grad=layer.params[key]['deriv'],
                                  sum_square=self.sum_squares[i][key])

    # Método específico de AdaGrad para actualizar los parámetros.
    def _update_rule(self, **kwargs) -> None:
        # Actualiza la suma acumulada de los cuadrados de los gradientes.
        kwargs['sum_square'] += self.eps + np.power(kwargs['grad'], 2)
        # Calcula la tasa de aprendizaje escalada.
        lr_scaled = self.lr / np.sqrt(kwargs['sum_square'])
        # Utiliza la tasa de aprendizaje escalada para actualizar los parámetros.
        kwargs['param'] -= lr_scaled * kwargs['grad']


#### Funciones de pérdida

In [ ]:
import numpy as np
def assert_same_shape(output: np.ndarray, output_grad: np.ndarray):
    assert output.shape == output_grad.shape, \
        '''
        Dos tensores deben tener la misma forma;
        en cambio, la forma del primer tensor es {0}
        y la forma del segundo tensor es {1}.
        '''.format(tuple(output_grad.shape), tuple(output.shape))
    return None



In [ ]:
class Loss(object):
    # Constructor de la clase base para las pérdidas.
    def __init__(self):
        pass

    # Método forward para calcular la pérdida entre las predicciones y los objetivos.
    def forward(self,
                prediction: ndarray,
                target: ndarray) -> float:
        # Asegura que las predicciones y los objetivos tengan la misma forma.
        assert_same_shape(prediction, target)

        self.prediction = prediction  # Almacena las predicciones.
        self.target = target  # Almacena los objetivos.

        self.output = self._output()  # Calcula la pérdida utilizando una función interna.

        return self.output  # Devuelve el valor de la pérdida.
    
    # Método backward para calcular el gradiente de la pérdida respecto a las entradas.
    def backward(self) -> ndarray:
        self.input_grad = self._input_grad()  # Calcula el gradiente utilizando una función interna.

        # Asegura que las predicciones y el gradiente tengan la misma forma.
        assert_same_shape(self.prediction, self.input_grad)

        return self.input_grad  # Devuelve el gradiente de la entrada.

    # Método abstracto para calcular la pérdida; debe ser implementado por subclases.
    def _output(self) -> float:
        raise NotImplementedError()

    # Método abstracto para calcular el gradiente de la entrada; debe ser implementado por subclases.
    def _input_grad(self) -> ndarray:
        raise NotImplementedError()


class SoftmaxCrossEntropy(Loss):
    # Constructor de la clase para la pérdida de entropía cruzada con softmax.
    def __init__(self, eps: float=1e-9) -> None:
        super().__init__()
        self.eps = eps  # Un pequeño número para evitar inestabilidad numérica.

    # Método para calcular la salida de la pérdida de entropía cruzada con softmax.
    def _output(self) -> float:
        out = []
        # Aplica softmax a cada fila de la predicción.
        for row in self.prediction:
            out.append(softmax(row, axis=1))
        softmax_preds = np.stack(out)

        # Recorta la salida de softmax para prevenir inestabilidad numérica.
        self.softmax_preds = np.clip(softmax_preds, self.eps, 1 - self.eps)

        # Calcula la pérdida real de entropía cruzada.
        softmax_cross_entropy_loss = -1.0 * self.target * np.log(self.softmax_preds) - \
            (1.0 - self.target) * np.log(1 - self.softmax_preds)

        return np.sum(softmax_cross_entropy_loss)  # Retorna la suma total de la pérdida.

    # Método para calcular el gradiente de la entrada basado en la salida de softmax.
    def _input_grad(self) -> np.ndarray:
        return self.softmax_preds - self.target  # Gradiente de la pérdida respecto a la entrada.


 ## Redes neuronales recurrentes
Una red neuronal recurrente (RNN) es una clase de redes neuronales diseñadas para manejar secuencias de datos, como series temporales o secuencias lingüísticas. 

A diferencia de las redes neuronales tradicionales que asumen independencia entre las entradas, las RNNs tienen "memoria" sobre entradas anteriores. Esto les permite retener información a través del tiempo y utilizar esta información para influir en la salida actual, lo cual es crucial para tareas donde el contexto y el orden de los datos son importantes, como el procesamiento del lenguaje natural o el análisis de series temporales.

#### Clase RNNNode
La clase RNNNode define un nodo dentro de una red neuronal recurrente (RNN), capaz de realizar cálculos hacia adelante y hacia atrás. El método forward calcula la salida y el nuevo estado oculto del nodo basado en la entrada actual y el estado oculto anterior, utilizando matrices de pesos y sesgos. 

El método backward se encarga de la retropropagación del error, calculando los gradientes respecto a las entradas y actualizando los parámetros del modelo basados en estos gradientes. Esto permite entrenar la red para ajustar sus pesos y mejorar la precisión de sus predicciones a lo largo del tiempo.

In [ ]:
class RNNNode(object):
    # Constructor del nodo RNN, inicialmente no realiza ninguna operación específica.
    def __init__(self):
        pass

    # Método forward para calcular la salida del nodo RNN a partir de la entrada y el estado oculto anterior.
    def forward(self,
                x_in: ndarray, 
                H_in: ndarray,
                params_dict: Dict[str, Dict[str, ndarray]]
                ) -> Tuple[ndarray]:
        '''
        Calcula la salida del nodo RNN y el nuevo estado oculto.
        Args:
        x_in: Arreglo de NumPy con forma (tamaño_lote, tam_vocabulario)
        H_in: Arreglo de NumPy con forma (tamaño_lote, tam_oculto)
        Retorna x_out: Arreglo de NumPy con forma (tamaño_lote, tam_vocabulario)
        Retorna H_out: Arreglo de NumPy con forma (tamaño_lote, tam_oculto)
        '''
        self.X_in = x_in
        self.H_in = H_in
    
        # Concatena la entrada x con el estado oculto anterior H.
        self.Z = np.column_stack((x_in, H_in))
        
        # Calcula el nuevo estado oculto intermedio usando los pesos y biases.
        self.H_int = np.dot(self.Z, params_dict['W_f']['value']) + params_dict['B_f']['value']
        
        # Aplica la función de activación tanh al estado oculto intermedio.
        self.H_out = tanh(self.H_int)

        # Calcula la salida del nodo RNN.
        self.X_out = np.dot(self.H_out, params_dict['W_v']['value']) + params_dict['B_v']['value']
        
        return self.X_out, self.H_out

    # Método backward para la propagación hacia atrás del error a través del nodo RNN.
    def backward(self, 
                 X_out_grad: ndarray, 
                 H_out_grad: ndarray,
                 params_dict: Dict[str, Dict[str, ndarray]]) -> Tuple[ndarray]:
        '''
        Retropropaga el gradiente a través del nodo RNN.
        Args:
        X_out_grad: Arreglo de NumPy con forma (tamaño_lote, tam_vocabulario)
        H_out_grad: Arreglo de NumPy con forma (tamaño_lote, tam_oculto)
        Retorna X_in_grad: Arreglo de NumPy con forma (tamaño_lote, tam_vocabulario)
        Retorna H_in_grad: Arreglo de NumPy con forma (tamaño_lote, tam_oculto)
        '''
        
        # Verifica que los gradientes y las salidas tengan la misma forma.
        assert_same_shape(X_out_grad, self.X_out)
        assert_same_shape(H_out_grad, self.H_out)

        # Calcula los gradientes para los parámetros de salida y acumula en los derivados.
        params_dict['B_v']['deriv'] += X_out_grad.sum(axis=0)
        params_dict['W_v']['deriv'] += np.dot(self.H_out.T, X_out_grad)
        
        # Propaga el gradiente hacia atrás a través de la red.
        dh = np.dot(X_out_grad, params_dict['W_v']['value'].T)
        dh += H_out_grad
        
        # Calcula el gradiente del estado oculto intermedio.
        dH_int = dh * dtanh(self.H_int)
        
        # Acumula gradientes en los parámetros del estado oculto.
        params_dict['B_f']['deriv'] += dH_int.sum(axis=0)
        params_dict['W_f']['deriv'] += np.dot(self.Z.T, dH_int)     
        
        # Calcula los gradientes de entrada.
        dz = np.dot(dH_int, params_dict['W_f']['value'].T)
        X_in_grad = dz[:, :self.X_in.shape[1]]
        H_in_grad = dz[:, self.X_in.shape[1]:]
        
        return X_in_grad, H_in_grad


#### RNNLayer

La clase RNNLayer es una capa de red neuronal recurrente que maneja la propagación hacia adelante y hacia atrás de los datos a través de una secuencia de tiempo. En el método forward, la capa procesa secuencialmente la entrada utilizando nodos RNN internos, cada uno correspondiente a un paso de tiempo, manteniendo un estado oculto que pasa de un nodo a otro. Esta capa es capaz de ajustar sus pesos y sesgos para mejorar la predicción del siguiente carácter en una secuencia.

Durante la retropropagación, calcula los gradientes para actualizar los parámetros con el fin de minimizar el error en las predicciones. Esta estructura es fundamental para tareas de procesamiento de secuencias como la generación de texto, donde la dependencia temporal entre los datos es crucial.

In [ ]:
class RNNLayer(object):
    # Constructor de la clase de la capa RNN.
    # Args:
    #   hidden_size: int - Número de neuronas ocultas en la capa RNN.
    #   output_size: int - Número de caracteres en el vocabulario para predecir el siguiente carácter.
    #   weight_scale: float - Escala para la inicialización de los pesos.
    def __init__(self,
                 hidden_size: int,
                 output_size: int,
                 weight_scale: float = None):
        self.hidden_size = hidden_size  # Almacena el tamaño del estado oculto.
        self.output_size = output_size  # Almacena el tamaño de salida.
        self.weight_scale = weight_scale  # Escala de inicialización de los pesos.
        self.start_H = np.zeros((1, hidden_size))  # Estado oculto inicial.
        self.first = True  # Bandera para inicialización en el primer paso forward.

    # Método para inicializar los parámetros de la capa.
    def _init_params(self, input_: ndarray):
        self.vocab_size = input_.shape[2]  # Tamaño del vocabulario a partir de la entrada.
        # Establece la escala de peso si no se proporcionó.
        if not self.weight_scale:
            self.weight_scale = 2 / (self.vocab_size + self.output_size)
        
        self.params = {'W_f': {}, 'B_f': {}, 'W_v': {}, 'B_v': {}}
        # Inicializa pesos y sesgos con valores aleatorios normalizados.
        self.params['W_f']['value'] = np.random.normal(0.0, self.weight_scale,
                                                       (self.hidden_size + self.vocab_size, self.hidden_size))
        self.params['B_f']['value'] = np.random.normal(0.0, self.weight_scale, (1, self.hidden_size))
        self.params['W_v']['value'] = np.random.normal(0.0, self.weight_scale,
                                                       (self.hidden_size, self.output_size))
        self.params['B_v']['value'] = np.random.normal(0.0, self.weight_scale, (1, self.output_size))

        self.params['W_f']['deriv'] = np.zeros_like(self.params['W_f']['value'])
        self.params['B_f']['deriv'] = np.zeros_like(self.params['B_f']['value'])
        self.params['W_v']['deriv'] = np.zeros_like(self.params['W_v']['value'])
        self.params['B_v']['deriv'] = np.zeros_like(self.params['B_v']['value'])
        
        self.cells = [RNNNode() for _ in range(input_.shape[1])]  # Inicializa nodos RNN por cada paso de secuencia.

    # Limpia los gradientes acumulados en los parámetros.
    def _clear_gradients(self):
        for key in self.params.keys():
            self.params[key]['deriv'] = np.zeros_like(self.params[key]['deriv'])

    # Procesa la entrada a través de la capa RNN y calcula la salida para cada paso de tiempo.
    def forward(self, x_seq_in: ndarray):
        if self.first:
            self._init_params(x_seq_in)  # Inicializa parámetros en el primer paso.
            self.first = False
        
        batch_size = x_seq_in.shape[0]
        H_in = np.repeat(self.start_H, batch_size, axis=0)
        sequence_length = x_seq_in.shape[1]
        x_seq_out = np.zeros((batch_size, sequence_length, self.output_size))
        
        for t in range(sequence_length):
            x_in = x_seq_in[:, t, :]
            y_out, H_in = self.cells[t].forward(x_in, H_in, self.params)
            x_seq_out[:, t, :] = y_out
    
        self.start_H = H_in.mean(axis=0, keepdims=True)  # Actualiza el estado oculto inicial para la próxima ejecución.
        
        return x_seq_out

    # Retropropaga el error desde la salida hacia las entradas.
    def backward(self, x_seq_out_grad: ndarray):
        batch_size = x_seq_out_grad.shape[0]
        h_in_grad = np.zeros((batch_size, self.hidden_size))
        sequence_length = x_seq_out_grad.shape[1]
        x_seq_in_grad = np.zeros((batch_size, sequence_length, self.vocab_size))
        
        for t in reversed(range(sequence_length)):
            x_out_grad = x_seq_out_grad[:, t, :]
            grad_out, h_in_grad = self.cells[t].backward(x_out_grad, h_in_grad, self.params)
            x_seq_in_grad[:, t, :] = grad_out
        
        return x_seq_in_grad


#### RNNModel

El código define un modelo de red neuronal recurrente (RNN) que es capaz de procesar secuencias de datos, como series temporales o texto. El modelo está compuesto por varias capas (RNNLayer), cada una procesando la entrada y pasándola a la siguiente. El proceso de entrenamiento ocurre en pasos, donde cada paso involucra:

* Paso hacia adelante (forward): Cada entrada de la secuencia es procesada por todas las capas de la red, pasando de una a otra. Esta operación se utiliza para obtener la salida de la red que luego se compara con el objetivo real para calcular la pérdida.
* Cálculo de la pérdida: Se usa un objeto de pérdida para evaluar qué tan bien la salida de la red coincide con los objetivos esperados.

* Paso hacia atrás (backward): Una vez calculada la pérdida, se calcula el gradiente de la pérdida respecto a las salidas, y este gradiente se propaga hacia atrás a través de la red para actualizar los pesos de las neuronas en cada capa, lo que permite que la red aprenda.

* Actualización de parámetros: Basándose en los gradientes obtenidos de la retropropagación, se actualizan los parámetros de la red.

In [ ]:
class RNNModel(object):
    '''
    Clase Modelo que recibe entradas y objetivos, entrena la red y calcula la pérdida.
    '''
    def __init__(self, 
                 layers: List[RNNLayer],
                 sequence_length: int, 
                 vocab_size: int, 
                 loss: Loss):
        '''
        Inicializa el modelo de red neuronal recurrente.
        Args:
        layers: Lista de capas RNN en la red.
        sequence_length: Longitud de la secuencia que pasa a través de la red.
        vocab_size: Número de caracteres en el vocabulario.
        loss: Objeto de pérdida utilizado para calcular la pérdida durante el entrenamiento.
        '''
        self.layers = layers  # Lista de capas RNN.
        self.vocab_size = vocab_size  # Tamaño del vocabulario.
        self.sequence_length = sequence_length  # Longitud de la secuencia.
        self.loss = loss  # Objeto de pérdida.
        # Establece la longitud de la secuencia para cada capa.
        for layer in self.layers:
            setattr(layer, 'sequence_length', sequence_length)

    def forward(self, 
                x_batch: ndarray):
        '''
        Realiza la propagación hacia adelante a través de la red.
        Args:
        x_batch: Array de entrada con forma (batch_size, sequence_length, vocab_size)
        Returns:
        x_batch_in: Array de salida de la última capa.
        '''       
        for layer in self.layers:
            x_batch = layer.forward(x_batch)  # Propaga la entrada a través de cada capa.
        return x_batch
        
    def backward(self, 
                 loss_grad: ndarray):
        '''
        Realiza la retropropagación a través de la red utilizando el gradiente de la pérdida.
        Args:
        loss_grad: Gradiente de la pérdida con forma (batch_size, sequence_length, vocab_size)
        Returns:
        loss_grad: Propaga el gradiente a través de todas las capas.
        '''
        for layer in reversed(self.layers):
            loss_grad = layer.backward(loss_grad)  # Retropropaga a través de cada capa en orden inverso.
        return loss_grad
                
    def single_step(self, 
                    x_batch: ndarray, 
                    y_batch: ndarray):
        '''
        Ejecuta un único paso de entrenamiento completo:
        1. Paso hacia adelante y aplicación de softmax.
        2. Calcula la pérdida y su gradiente.
        3. Paso hacia atrás.
        4. Actualización de parámetros.
        Args:
        x_batch: Array de entrada con forma (batch_size, sequence_length, vocab_size)
        y_batch: Array objetivo correspondiente.
        Returns:
        loss: Pérdida calculada para el lote actual.
        '''
        x_batch_out = self.forward(x_batch)  # Paso hacia adelante.
        loss = self.loss.forward(x_batch_out, y_batch)  # Calcula la pérdida.
        loss_grad = self.loss.backward()  # Calcula el gradiente de la pérdida.
        for layer in self.layers:
            layer._clear_gradients()  # Limpia los gradientes en cada capa.
        self.backward(loss_grad)  # Retropropaga el gradiente de la pérdida.
        return loss


#### RNNTrainer

Este código define una clase RNNTrainer que se utiliza para entrenar un modelo de red neuronal recurrente (RNN) para la generación de texto. Utiliza un archivo de texto como datos de entrada y realiza las siguientes tareas principales:

* Inicialización: Prepara el modelo, el optimizador y los datos necesarios para el entrenamiento, incluyendo la creación de mapeos de caracteres a índices y viceversa.

* Generación de entradas y objetivos: Crea los lotes de datos de entrada y los objetivos (targets) correspondientes que el modelo intentará predecir.

* Entrenamiento: Ejecuta el proceso de entrenamiento en varias iteraciones, donde cada iteración incluye un paso hacia adelante (forward), el cálculo de la pérdida, un paso hacia atrás (backward) para la propagación del error, y la actualización de los parámetros del modelo.

* Muestreo de salidas: Opcionalmente, genera texto basado en el modelo entrenado para visualizar cómo está aprendiendo el modelo durante el entrenamiento.

In [ ]:
class RNNTrainer:
    '''
    Clase que toma un archivo de texto y un modelo, y comienza a generar caracteres.
    '''
    def __init__(self, 
                 text_file: str, 
                 model: RNNModel,
                 optim: RNNOptimizer,
                 batch_size: int = 32):
        # Leer los datos del archivo de texto.
        self.data = open(text_file, 'r').read()
        self.model = model  # Modelo de red neuronal recurrente.
        self.chars = list(set(self.data))  # Lista de caracteres únicos en el texto.
        self.vocab_size = len(self.chars)  # Tamaño del vocabulario.
        self.char_to_idx = {ch:i for i,ch in enumerate(self.chars)}  # Diccionario de caracteres a índices.
        self.idx_to_char = {i:ch for i,ch in enumerate(self.chars)}  # Diccionario inverso de índices a caracteres.
        self.sequence_length = self.model.sequence_length  # Longitud de la secuencia usada en el modelo.
        self.batch_size = batch_size  # Tamaño del lote para el entrenamiento.
        self.optim = optim  # Optimizador para ajustar los parámetros del modelo.
        setattr(self.optim, 'model', self.model)  # Establece el modelo en el optimizador.

    def _generate_inputs_targets(self, start_pos: int):
        # Genera índices para los lotes de entradas y objetivos desde una posición inicial.
        inputs_indices = np.zeros((self.batch_size, self.sequence_length), dtype=int)
        targets_indices = np.zeros((self.batch_size, self.sequence_length), dtype=int)
        
        for i in range(self.batch_size):
            inputs_indices[i, :] = np.array([self.char_to_idx[ch] 
                            for ch in self.data[start_pos + i: start_pos + self.sequence_length  + i]])
            targets_indices[i, :] = np.array([self.char_to_idx[ch] 
                         for ch in self.data[start_pos + 1 + i: start_pos + self.sequence_length + 1 + i]])

        return inputs_indices, targets_indices

    def _generate_one_hot_array(self, indices: ndarray):
        # Convierte los índices de caracteres a una representación one-hot.
        batch = []
        for seq in indices:
            one_hot_sequence = np.zeros((self.sequence_length, self.vocab_size))
            for i in range(self.sequence_length):
                one_hot_sequence[i, seq[i]] = 1.0
            batch.append(one_hot_sequence) 
        return np.stack(batch)

    def sample_output(self, input_char: int, sample_length: int):
        # Genera una muestra de salida del modelo actual, caracter por caracter.
        indices = []
        sample_model = deepcopy(self.model)  # Hace una copia del modelo para usar en muestreo.
        for i in range(sample_length):
            input_char_batch = np.zeros((1, 1, self.vocab_size))
            input_char_batch[0, 0, input_char] = 1.0
            x_batch_out = sample_model.forward(input_char_batch)
            x_softmax = batch_softmax(x_batch_out)
            input_char = np.random.choice(range(self.vocab_size), p=x_softmax.ravel())
            indices.append(input_char)
        txt = ''.join(self.idx_to_char[idx] for idx in indices)
        return txt

    def train(self, num_iterations: int, sample_every: int=100):
        # Entrena el modelo para generar caracteres.
        plot_iter = np.zeros((0))
        plot_loss = np.zeros((0))
        num_iter = 0
        start_pos = 0
        moving_average = deque(maxlen=100)
        while num_iter < num_iterations:
            if start_pos + self.sequence_length + self.batch_size + 1 > len(self.data):
                start_pos = 0
            inputs_indices, targets_indices = self._generate_inputs_targets(start_pos)
            inputs_batch, targets_batch = \
                self._generate_one_hot_array(inputs_indices), self._generate_one_hot_array(targets_indices)
            loss = self.model.single_step(inputs_batch, targets_batch)
            self.optim.step()
            moving_average.append(loss)
            ma_loss = np.mean(moving_average)
            start_pos += self.batch_size
            plot_iter = np.append(plot_iter, [num_iter])
            plot_loss = np.append(plot_loss, [ma_loss])
            if num_iter % sample_every == 0:
                plt.plot(plot_iter, plot_loss)
                display.clear_output(wait=True)
                plt.show()
                sample_text = self.sample_output(self.char_to_idx[self.data[start_pos]], 200)
                print(sample_text)
            num_iter += 1


#### Experimento

In [ ]:
capas = [RNNLayer(hidden_size=256, output_size=62)]
mod = RNNModel(layers=capas,
               vocab_size=62, sequence_length=10,
               loss=SoftmaxCrossEntropy())
optim = SGD(lr=0.001, gradient_clipping=True)
trainer = RNNTrainer('Ejemplo.txt', mod, optim)
trainer.train(100, sample_every=100)

### LSTM

**Clase LSTMNode**

- Clase `LSTMNode`: Representa un nodo en una capa de LSTM, encargado de procesar datos secuenciales.
- Método `forward`: Implementa el paso hacia adelante. Realiza cálculos para las puertas de la LSTM (olvido, entrada y salida) y actualiza el estado de memoria y el estado oculto.
- Método `backward`: Realiza el paso hacia atrás o retropropagación para ajustar los gradientes de los parámetros. Calcula los gradientes para cada puerta y los propaga hacia atrás, permitiendo actualizar los parámetros de la red LSTM.

In [ ]:
class LSTMNode:

    def __init__(self):
        '''
        Constructor de la clase LSTMNode. 
        Inicializa los parámetros que se utilizarán en el nodo de la LSTM.
        
        param hidden_size: int - número de neuronas ocultas en la capa LSTM.
        param vocab_size: int - tamaño del vocabulario, es decir, el número de caracteres o palabras posibles.
        '''
        pass

    def forward(self, 
                X_in: ndarray, 
                H_in: ndarray, 
                C_in: ndarray, 
                params_dict: Dict[str, Dict[str, ndarray]]):
        '''
        Realiza el paso hacia adelante de la LSTM.
        
        param X_in: numpy array con forma (batch_size, vocab_size), representa la entrada al nodo LSTM.
        param H_in: numpy array con forma (batch_size, hidden_size), representa el estado oculto anterior.
        param C_in: numpy array con forma (batch_size, hidden_size), representa el estado de memoria anterior.
        param params_dict: Diccionario que contiene los pesos y sesgos de la LSTM.
        
        return self.X_out: numpy array con forma (batch_size, output_size), salida de la red.
        return self.H_out: numpy array con forma (batch_size, hidden_size), nuevo estado oculto.
        return self.C_out: numpy array con forma (batch_size, hidden_size), nuevo estado de memoria.
        '''
        # Guarda la entrada y el estado de memoria anterior.
        self.X_in = X_in
        self.C_in = C_in

        # Concatenación de la entrada y el estado oculto para el cálculo de las puertas.
        self.Z = np.column_stack((X_in, H_in))
        
        # Cálculo de la puerta de olvido (forget gate).
        self.f_int = np.dot(self.Z, params_dict['W_f']['value']) + params_dict['B_f']['value']
        self.f = sigmoid(self.f_int)
        
        # Cálculo de la puerta de entrada (input gate).
        self.i_int = np.dot(self.Z, params_dict['W_i']['value']) + params_dict['B_i']['value']
        self.i = sigmoid(self.i_int)
        self.C_bar_int = np.dot(self.Z, params_dict['W_c']['value']) + params_dict['B_c']['value']
        self.C_bar = tanh(self.C_bar_int)

        # Cálculo del nuevo estado de memoria.
        self.C_out = self.f * C_in + self.i * self.C_bar
        
        # Cálculo de la puerta de salida (output gate).
        self.o_int = np.dot(self.Z, params_dict['W_o']['value']) + params_dict['B_o']['value']
        self.o = sigmoid(self.o_int)
        
        # Cálculo del nuevo estado oculto.
        self.H_out = self.o * tanh(self.C_out)

        # Cálculo de la salida de la red.
        self.X_out = np.dot(self.H_out, params_dict['W_v']['value']) + params_dict['B_v']['value']
        
        return self.X_out, self.H_out, self.C_out 

    def backward(self, 
                 X_out_grad: ndarray, 
                 H_out_grad: ndarray, 
                 C_out_grad: ndarray, 
                 params_dict: Dict[str, Dict[str, ndarray]]):
        '''
        Realiza el paso hacia atrás (backpropagation) de la LSTM.
        
        param X_out_grad: Gradiente de la pérdida con respecto a la salida.
        param H_out_grad: Gradiente de la pérdida con respecto al estado oculto.
        param C_out_grad: Gradiente de la pérdida con respecto al estado de memoria.
        param params_dict: Diccionario que contiene los pesos, sesgos y sus derivadas para actualizar los parámetros.
        
        return dx_prev: numpy array con forma (1, vocab_size), gradiente con respecto a la entrada anterior.
        return dH_prev: numpy array con forma (1, hidden_size), gradiente con respecto al estado oculto anterior.
        return dC_prev: numpy array con forma (1, hidden_size), gradiente con respecto al estado de memoria anterior.
        '''
        
        # Asegura que las formas de los gradientes coinciden con las salidas.
        assert_same_shape(X_out_grad, self.X_out)
        assert_same_shape(H_out_grad, self.H_out)
        assert_same_shape(C_out_grad, self.C_out)

        # Cálculo de las derivadas para los parámetros de la salida.
        params_dict['W_v']['deriv'] += np.dot(self.H_out.T, X_out_grad)
        params_dict['B_v']['deriv'] += X_out_grad.sum(axis=0)

        # Gradiente con respecto al estado oculto.
        dh_out = np.dot(X_out_grad, params_dict['W_v']['value'].T)        
        dh_out += H_out_grad
                         
        # Gradiente de la puerta de salida.
        do = dh_out * tanh(self.C_out)
        do_int = dsigmoid(self.o_int) * do
        params_dict['W_o']['deriv'] += np.dot(self.Z.T, do_int)
        params_dict['B_o']['deriv'] += do_int.sum(axis=0)

        # Gradiente del estado de memoria.
        dC_out = dh_out * self.o * dtanh(self.C_out)
        dC_out += C_out_grad
        dC_bar = dC_out * self.i
        dC_bar_int = dtanh(self.C_bar_int) * dC_bar
        params_dict['W_c']['deriv'] += np.dot(self.Z.T, dC_bar_int)
        params_dict['B_c']['deriv'] += dC_bar_int.sum(axis=0)

        # Gradiente de la puerta de entrada.
        di = dC_out * self.C_bar
        di_int = dsigmoid(self.i_int) * di
        params_dict['W_i']['deriv'] += np.dot(self.Z.T, di_int)
        params_dict['B_i']['deriv'] += di_int.sum(axis=0)

        # Gradiente de la puerta de olvido.
        df = dC_out * self.C_in
        df_int = dsigmoid(self.f_int) * df
        params_dict['W_f']['deriv'] += np.dot(self.Z.T, df_int)
        params_dict['B_f']['deriv'] += df_int.sum(axis=0)

        # Cálculo del gradiente con respecto a la entrada y al estado oculto.
        dz = (np.dot(df_int, params_dict['W_f']['value'].T)
             + np.dot(di_int, params_dict['W_i']['value'].T)
             + np.dot(dC_bar_int, params_dict['W_c']['value'].T)
             + np.dot(do_int, params_dict['W_o']['value'].T))
    
        dx_prev = dz[:, :self.X_in.shape[1]]
        dH_prev = dz[:, self.X_in.shape[1]:]
        dC_prev = self.f * dC_out

        return dx_prev, dH_prev, dC_prev


**LSTMLayer**

- Clase `LSTMLayer`: Representa una capa completa de nodos LSTM que procesa secuencias de datos.
- Método `__init__`: Configura los parámetros iniciales y los estados ocultos y de memoria.
- Método `_init_params`: Inicializa los parámetros de los pesos y sesgos de cada puerta (olvido, entrada, memoria, y salida) y los gradientes.
- Método `_clear_gradients`: Reinicia los gradientes a cero para evitar acumulación no deseada.
- Método `forward`: Ejecuta la secuencia de entrada a través de los nodos LSTM en orden, actualizando los estados ocultos y de memoria en cada paso de tiempo.
    Método `backward`: Ejecuta el paso hacia atrás para propagar los gradientes en orden inverso a través de cada paso de tiempo, actualizando los gradientes de los parámetros y acumulando el gradiente de la entrada.

In [ ]:
class LSTMLayer:

    def __init__(self,
                 hidden_size: int,
                 output_size: int,
                 weight_scale: float = 0.01):
        '''
        Constructor de la clase LSTMLayer. 
        Inicializa los parámetros y variables de estado de la capa LSTM.

        param hidden_size: int - número de neuronas ocultas en la capa LSTM.
        param output_size: int - tamaño de la salida de la red.
        param weight_scale: float - escala para inicializar los pesos.
        '''
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.weight_scale = weight_scale
        self.start_H = np.zeros((1, hidden_size))  # Estado oculto inicial
        self.start_C = np.zeros((1, hidden_size))  # Estado de memoria inicial        
        self.first = True  # Bandera para inicializar parámetros la primera vez

    def _init_params(self, input_: ndarray):
        '''
        Inicializa los parámetros de la capa LSTM.
        
        param input_: ndarray - entrada de la secuencia de datos con forma (batch_size, sequence_length, vocab_size)
        '''
        # Determina el tamaño del vocabulario basado en la entrada
        self.vocab_size = input_.shape[2]

        # Diccionario para almacenar los parámetros de las puertas de la LSTM
        self.params = {
            'W_f': {}, 'B_f': {}, 'W_i': {}, 'B_i': {}, 'W_c': {}, 'B_c': {},
            'W_o': {}, 'B_o': {}, 'W_v': {}, 'B_v': {}
        }
        
        # Inicializa los pesos y sesgos para cada puerta
        self.params['W_f']['value'] = np.random.normal(0.0, self.weight_scale,
                                                       (self.hidden_size + self.vocab_size, self.hidden_size))
        self.params['B_f']['value'] = np.random.normal(0.0, self.weight_scale,
                                                       (1, self.hidden_size))
        self.params['W_i']['value'] = np.random.normal(0.0, self.weight_scale,
                                                       (self.hidden_size + self.vocab_size, self.hidden_size))
        self.params['B_i']['value'] = np.random.normal(0.0, self.weight_scale,
                                                      (1, self.hidden_size))
        self.params['W_c']['value'] = np.random.normal(0.0, self.weight_scale,
                                                      (self.hidden_size + self.vocab_size, self.hidden_size))
        self.params['B_c']['value'] = np.random.normal(0.0, self.weight_scale,
                                                      (1, self.hidden_size))
        self.params['W_o']['value'] = np.random.normal(0.0, self.weight_scale,
                                                      (self.hidden_size + self.vocab_size, self.hidden_size))
        self.params['B_o']['value'] = np.random.normal(0.0, self.weight_scale,
                                                      (1, self.hidden_size))       
        self.params['W_v']['value'] = np.random.normal(0.0, self.weight_scale,
                                                      (self.hidden_size, self.output_size))
        self.params['B_v']['value'] = np.random.normal(0.0, self.weight_scale,
                                                      (1, self.output_size))
        
        # Inicializa los gradientes de los parámetros
        for key in self.params.keys():
            self.params[key]['deriv'] = np.zeros_like(self.params[key]['value'])
        
        # Crea una lista de nodos LSTM para cada paso de tiempo en la secuencia
        self.cells = [LSTMNode() for x in range(input_.shape[1])]

    def _clear_gradients(self):
        '''
        Reinicia los gradientes a cero para evitar acumulaciones no deseadas entre épocas de entrenamiento.
        '''
        for key in self.params.keys():
            self.params[key]['deriv'] = np.zeros_like(self.params[key]['deriv'])
                    
    def forward(self, x_seq_in: ndarray):
        '''
        Realiza el paso hacia adelante para la secuencia de entrada.

        param x_seq_in: numpy array con forma (batch_size, sequence_length, vocab_size)
        return x_seq_out: numpy array con forma (batch_size, sequence_length, output_size)
        '''
        if self.first:
            self._init_params(x_seq_in)  # Inicializa parámetros en la primera ejecución
            self.first = False
        
        batch_size = x_seq_in.shape[0]
        
        # Inicializa el estado oculto y de memoria para cada batch
        H_in = np.repeat(self.start_H, batch_size, axis=0)
        C_in = np.repeat(self.start_C, batch_size, axis=0)        

        sequence_length = x_seq_in.shape[1]
        
        # Inicializa la salida para cada paso de la secuencia
        x_seq_out = np.zeros((batch_size, sequence_length, self.output_size))
        
        # Itera sobre cada paso de tiempo en la secuencia
        for t in range(sequence_length):
            x_in = x_seq_in[:, t, :]  # Entrada en el paso de tiempo t
            
            # Propaga el paso hacia adelante en el nodo LSTM correspondiente
            y_out, H_in, C_in = self.cells[t].forward(x_in, H_in, C_in, self.params)
      
            x_seq_out[:, t, :] = y_out  # Guarda la salida en la secuencia de salida
    
        # Actualiza los estados iniciales para el próximo batch
        self.start_H = H_in.mean(axis=0, keepdims=True)
        self.start_C = C_in.mean(axis=0, keepdims=True)        
        
        return x_seq_out

    def backward(self, x_seq_out_grad: ndarray):
        '''
        Realiza el paso hacia atrás (backpropagation) para la secuencia de salida.

        param x_seq_out_grad: numpy array con forma (batch_size, sequence_length, output_size)
        return x_seq_in_grad: numpy array con forma (batch_size, sequence_length, vocab_size)
        '''
        
        batch_size = x_seq_out_grad.shape[0]
        
        # Inicializa los gradientes del estado oculto y del estado de memoria
        h_in_grad = np.zeros((batch_size, self.hidden_size))
        c_in_grad = np.zeros((batch_size, self.hidden_size))        
        
        num_chars = x_seq_out_grad.shape[1]
        
        # Inicializa el gradiente de la entrada de la secuencia
        x_seq_in_grad = np.zeros((batch_size, num_chars, self.vocab_size))
        
        # Itera en reversa a través de cada paso de tiempo en la secuencia
        for t in reversed(range(num_chars)):
            x_out_grad = x_seq_out_grad[:, t, :]  # Gradiente de la salida en el paso t

            # Propaga el paso hacia atrás en el nodo LSTM correspondiente
            grad_out, h_in_grad, c_in_grad = \
                self.cells[t].backward(x_out_grad, h_in_grad, c_in_grad, self.params)
        
            x_seq_in_grad[:, t, :] = grad_out  # Guarda el gradiente de la entrada en el paso t
        
        return x_seq_in_grad


**LSTMModel**

- Clase `LSTMModel`: Define el modelo de red LSTM que incluye múltiples capas y se encarga de entrenar el modelo y calcular la pérdida.
- Método `__init__`: Inicializa el modelo, asigna las capas y la función de pérdida, y establece la longitud de secuencia en cada capa.
- Método `forward`: Realiza el paso hacia adelante a través de todas las capas, generando la salida final del modelo.
- Método `backward`: Realiza el paso hacia atrás para propagar los gradientes de error a través de todas las capas en orden inverso, actualizando gradientes de cada capa.
- Método `single_step`: Realiza un solo paso de entrenamiento, incluyendo el paso hacia adelante, cálculo de la pérdida, propagación hacia atrás, y la actualización de parámetros en cada capa.

In [ ]:
class LSTMModel(object):
    '''
    Clase del modelo que recibe las entradas y objetivos, entrena la red y calcula la pérdida.
    '''
    def __init__(self, 
                 layers: List[LSTMLayer],
                 sequence_length: int, 
                 vocab_size: int, 
                 hidden_size: int,
                 loss: Loss):
        '''
        Inicializa el modelo con las capas, el tamaño de la secuencia, el vocabulario y la función de pérdida.

        param layers: List[LSTMLayer] - lista de capas LSTM que forman la red.
        param sequence_length: int - longitud de la secuencia que se pasa a través de la red.
        param vocab_size: int - número de caracteres en el vocabulario.
        param hidden_size: int - número de neuronas ocultas en cada capa.
        param loss: Loss - objeto que representa la función de pérdida.
        '''
        self.layers = layers
        self.vocab_size = vocab_size
        self.hidden_size = hidden_size
        self.sequence_length = sequence_length
        self.loss = loss
        
        # Asigna la longitud de la secuencia a cada capa
        for layer in self.layers:
            setattr(layer, 'sequence_length', sequence_length)

    def forward(self, x_batch: ndarray):
        '''
        Realiza el paso hacia adelante para todo el modelo, propagando la entrada a través de cada capa.

        param x_batch: numpy array con forma (batch_size, sequence_length, vocab_size)
        returns x_batch: numpy array con forma (batch_size, sequence_length, vocab_size), la salida de la red.
        '''       
        for layer in self.layers:
            x_batch = layer.forward(x_batch)  # Pasa la salida de cada capa como entrada a la siguiente capa
                
        return x_batch
        
    def backward(self, loss_grad: ndarray):
        '''
        Realiza el paso hacia atrás para todo el modelo, propagando el gradiente de pérdida hacia atrás.

        param loss_grad: numpy array con forma (batch_size, sequence_length, vocab_size)
        returns loss_grad: numpy array, el gradiente de pérdida tras propagarse a través de las capas.
        '''
        for layer in reversed(self.layers):  # Itera en orden inverso sobre las capas
            loss_grad = layer.backward(loss_grad)  # Pasa el gradiente hacia atrás en cada capa
            
        return loss_grad
                
    def single_step(self, x_batch: ndarray, y_batch: ndarray):
        '''
        Ejecuta un solo paso de entrenamiento:
        1. Paso hacia adelante y cálculo de softmax
        2. Computa la pérdida y el gradiente de la pérdida
        3. Paso hacia atrás (backpropagation)
        4. Actualiza los parámetros de las capas

        param x_batch: ndarray - entrada de la secuencia con forma (batch_size, sequence_length, vocab_size)
        param y_batch: ndarray - objetivos de la secuencia con la misma forma que x_batch
        return loss: float - pérdida calculada en este paso
        '''  
        # Paso hacia adelante
        x_batch_out = self.forward(x_batch)
        
        # Calcula la pérdida entre la salida y las etiquetas objetivo
        loss = self.loss.forward(x_batch_out, y_batch)
        
        # Calcula el gradiente de la pérdida
        loss_grad = self.loss.backward()
        
        # Limpia los gradientes acumulados en cada capa
        for layer in self.layers:
            layer._clear_gradients()
        
        # Paso hacia atrás para actualizar los gradientes en cada capa
        self.backward(loss_grad)
        
        return loss


### GRU

**GRUNode**

- Clase `GRUNode`: Representa un nodo de una capa GRU, encargado de calcular los estados internos de una red GRU para cada paso temporal en una secuencia de entrada.
- Método `__init__`: Inicializa el nodo sin configurar ningún parámetro en el constructor.
- Método `forward`: Realiza el paso hacia adelante (forward) del nodo GRU, calculando las puertas de reinicio y actualización, así como el nuevo estado oculto. La salida final del nodo depende de una combinación de la entrada y el estado oculto anterior.
- Método `backward`: Realiza el paso hacia atrás (backpropagation) para actualizar los gradientes de los parámetros de la GRU, pasando los gradientes hacia atrás para ajustar los pesos de cada puerta y los estados de salida.


In [ ]:
class GRUNode(object):

    def __init__(self):
        '''
        Inicializa un nodo GRU sin definir parámetros en el constructor.
        
        param hidden_size: int - número de neuronas ocultas en la capa de GRU.
        param vocab_size: int - tamaño del vocabulario para la predicción del próximo carácter.
        '''
        pass
        
    def forward(self, 
                X_in: ndarray, 
                H_in: ndarray,
                params_dict: Dict[str, Dict[str, ndarray]]) -> Tuple[ndarray]:
        '''
        Realiza el paso hacia adelante del nodo GRU.

        param X_in: numpy array con forma (batch_size, vocab_size), representa la entrada actual.
        param H_in: numpy array con forma (batch_size, hidden_size), representa el estado oculto anterior.
        param params_dict: Diccionario que contiene los pesos y sesgos.
        
        return self.X_out: numpy array con forma (batch_size, vocab_size), la salida de la red.
        return self.H_out: numpy array con forma (batch_size, hidden_size), el nuevo estado oculto.
        '''
        self.X_in = X_in
        self.H_in = H_in        
        
        # Cálculo de la puerta de reinicio (reset gate)
        self.X_r = np.dot(X_in, params_dict['W_xr']['value'])
        self.H_r = np.dot(H_in, params_dict['W_hr']['value'])

        # Cálculo de la puerta de actualización (update gate)        
        self.X_u = np.dot(X_in, params_dict['W_xu']['value'])
        self.H_u = np.dot(H_in, params_dict['W_hu']['value'])        
        
        # Aplicación de las funciones de activación para las puertas
        self.r_int = self.X_r + self.H_r + params_dict['B_r']['value']
        self.r = sigmoid(self.r_int)
        
        self.u_int = self.X_r + self.H_r + params_dict['B_u']['value']
        self.u = sigmoid(self.u_int)

        # Cálculo del nuevo estado
        self.h_reset = self.r * H_in
        self.X_h = np.dot(X_in, params_dict['W_xh']['value'])
        self.H_h = np.dot(self.h_reset, params_dict['W_hh']['value']) 
        self.h_bar_int = self.X_h + self.H_h + params_dict['B_h']['value']
        self.h_bar = tanh(self.h_bar_int)        
        
        # Cálculo del estado oculto de salida
        self.H_out = self.u * self.H_in + (1 - self.u) * self.h_bar

        # Cálculo de la salida de la red
        self.X_out = np.dot(self.H_out, params_dict['W_v']['value']) + params_dict['B_v']['value']
        
        return self.X_out, self.H_out


    def backward(self, 
                 X_out_grad: ndarray, 
                 H_out_grad: ndarray, 
                 params_dict: Dict[str, Dict[str, ndarray]]):
        '''
        Realiza el paso hacia atrás (backpropagation) para el nodo GRU.

        param X_out_grad: Gradiente de la pérdida respecto a la salida.
        param H_out_grad: Gradiente de la pérdida respecto al estado oculto.
        param params_dict: Diccionario con los pesos y sus derivadas para actualizar los parámetros.
        
        return dX_in: Gradiente con respecto a la entrada X_in.
        return dH_in: Gradiente con respecto al estado oculto H_in.
        '''
        
        # Gradiente de la capa de salida
        params_dict['B_v']['deriv'] += X_out_grad.sum(axis=0)
        params_dict['W_v']['deriv'] += np.dot(self.H_out.T, X_out_grad)

        # Gradiente con respecto al estado oculto
        dh_out = np.dot(X_out_grad, params_dict['W_v']['value'].T)        
        dh_out += H_out_grad
                         
        # Gradiente de la puerta de actualización
        du = self.H_in * H_out_grad - self.h_bar * H_out_grad 
        dh_bar = (1 - self.u) * H_out_grad
        
        dh_bar_int = dh_bar * dtanh(self.h_bar_int)
        params_dict['B_h']['deriv'] += dh_bar_int.sum(axis=0)
        params_dict['W_xh']['deriv'] += np.dot(self.X_in.T, dh_bar_int)
        
        dX_in = np.dot(dh_bar_int, params_dict['W_xh']['value'].T)
 
        params_dict['W_hh']['deriv'] += np.dot(self.h_reset.T, dh_bar_int)
        dh_reset = np.dot(dh_bar_int, params_dict['W_hh']['value'].T)   
        
        # Gradiente de la puerta de reinicio
        dr = dh_reset * self.H_in
        dH_in = dh_reset * self.r        
        
        # Rama de actualización
        du_int = dsigmoid(self.u_int) * du
        params_dict['B_u']['deriv'] += du_int.sum(axis=0)

        dX_in += np.dot(du_int, params_dict['W_xu']['value'].T)
        params_dict['W_xu']['deriv'] += np.dot(self.X_in.T, du_int)
        
        dH_in += np.dot(du_int, params_dict['W_hu']['value'].T)
        params_dict['W_hu']['deriv'] += np.dot(self.H_in.T, du_int)        

        # Rama de reinicio
        dr_int = dsigmoid(self.r_int) * dr
        params_dict['B_r']['deriv'] += dr_int.sum(axis=0)

        dX_in += np.dot(dr_int, params_dict['W_xr']['value'].T)
        params_dict['W_xr']['deriv'] += np.dot(self.X_in.T, dr_int)
        
        dH_in += np.dot(dr_int, params_dict['W_hr']['value'].T)
        params_dict['W_hr']['deriv'] += np.dot(self.H_in.T, dr_int)   
        
        return dX_in, dH_in


**GRULayer**

- Clase `GRULayer`: Define una capa GRU compuesta de múltiples nodos GRU, que se encargan de procesar una secuencia de datos en varias etapas de tiempo.
- Método `__init__`: Configura los parámetros iniciales de la capa, incluyendo el tamaño de la salida, la escala de pesos, y el estado oculto inicial.
- Método `_init_params`: Inicializa los pesos y sesgos de cada puerta en la capa GRU para la entrada y el estado oculto. Los pesos están asociados a las puertas de reinicio, actualización y salida de cada nodo.
- Método `_clear_gradients`: Reinicia los gradientes de cada parámetro de la capa a cero para evitar acumulaciones.
- Método `forward`: Realiza el paso hacia adelante en toda la capa GRU, procesando la secuencia de entrada. En cada paso de tiempo, actualiza el estado oculto y almacena la salida correspondiente.
- Método `backward`: Realiza el paso hacia atrás para calcular los gradientes de la pérdida y actualiza los gradientes de los parámetros para cada nodo en la secuencia.

In [ ]:
class GRULayer(object):

    def __init__(self,
                 hidden_size: int,
                 output_size: int,
                 weight_scale: float = 0.01):
        '''
        Inicializa una capa de nodos GRU.

        param hidden_size: int - número de neuronas ocultas en la capa GRU.
        param output_size: int - tamaño de la salida de la capa GRU.
        param weight_scale: float - escala para inicializar los pesos aleatoriamente.
        '''
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.weight_scale = weight_scale
        self.start_H = np.zeros((1, hidden_size))  # Estado inicial oculto
        self.first = True  # Indica si es la primera vez que se ejecuta el forward para inicializar parámetros

        
    def _init_params(self, input_: ndarray):
        '''
        Inicializa los parámetros de la capa GRU.

        param input_: ndarray - entrada de la secuencia con forma (batch_size, sequence_length, vocab_size).
        '''
        
        self.vocab_size = input_.shape[2]

        # Diccionario que almacena los pesos y sesgos para cada puerta en la GRU
        self.params = {
            'W_xr': {}, 'W_hr': {}, 'B_r': {},
            'W_xu': {}, 'W_hu': {}, 'B_u': {},
            'W_xh': {}, 'W_hh': {}, 'B_h': {},
            'W_v': {}, 'B_v': {}
        }
        
        # Inicialización de los pesos y sesgos para las puertas de reinicio, actualización y salida
        self.params['W_xr']['value'] = np.random.normal(0.0, self.weight_scale, (self.vocab_size, self.hidden_size))
        self.params['W_hr']['value'] = np.random.normal(0.0, self.weight_scale, (self.hidden_size, self.hidden_size))
        self.params['B_r']['value'] = np.random.normal(0.0, self.weight_scale, (1, self.hidden_size))
        
        self.params['W_xu']['value'] = np.random.normal(0.0, self.weight_scale, (self.vocab_size, self.hidden_size))
        self.params['W_hu']['value'] = np.random.normal(0.0, self.weight_scale, (self.hidden_size, self.hidden_size))
        self.params['B_u']['value'] = np.random.normal(0.0, self.weight_scale, (1, self.hidden_size))
        
        self.params['W_xh']['value'] = np.random.normal(0.0, self.weight_scale, (self.vocab_size, self.hidden_size))
        self.params['W_hh']['value'] = np.random.normal(0.0, self.weight_scale, (self.hidden_size, self.hidden_size))
        self.params['B_h']['value'] = np.random.normal(0.0, 1.0, (1, self.hidden_size))
        
        self.params['W_v']['value'] = np.random.normal(0.0, 1.0, (self.hidden_size, self.output_size))
        self.params['B_v']['value'] = np.random.normal(0.0, 1.0, (1, self.output_size))    

        # Inicialización de los gradientes para cada parámetro
        for key in self.params.keys():
            self.params[key]['deriv'] = np.zeros_like(self.params[key]['value'])
        
        # Crea una lista de nodos GRU, uno por cada paso de tiempo en la secuencia de entrada
        self.cells = [GRUNode() for x in range(input_.shape[1])]


    def _clear_gradients(self):
        '''
        Reinicia los gradientes a cero para evitar acumulación no deseada entre lotes (batches).
        '''
        for key in self.params.keys():
            self.params[key]['deriv'] = np.zeros_like(self.params[key]['deriv'])
                    
        
    def forward(self, x_seq_in: ndarray):
        '''
        Realiza el paso hacia adelante en la capa GRU para una secuencia de entrada.

        param x_seq_in: numpy array con forma (batch_size, sequence_length, vocab_size)
        return x_seq_out: numpy array con forma (batch_size, sequence_length, output_size)
        '''
        if self.first:
            self._init_params(x_seq_in)  # Inicializa los parámetros en la primera ejecución
            self.first = False
        
        batch_size = x_seq_in.shape[0]
        
        # Inicializa el estado oculto para cada lote
        H_in = np.repeat(self.start_H, batch_size, axis=0)

        sequence_length = x_seq_in.shape[1]
        
        # Inicializa la salida para cada paso de la secuencia
        x_seq_out = np.zeros((batch_size, sequence_length, self.output_size))
        
        # Itera sobre cada paso de tiempo en la secuencia
        for t in range(sequence_length):
            x_in = x_seq_in[:, t, :]  # Entrada en el paso de tiempo t
            
            # Ejecuta el paso hacia adelante en el nodo GRU correspondiente
            y_out, H_in = self.cells[t].forward(x_in, H_in, self.params)
      
            x_seq_out[:, t, :] = y_out  # Guarda la salida en la secuencia de salida
    
        # Actualiza el estado inicial para el próximo lote
        self.start_H = H_in.mean(axis=0, keepdims=True)
        
        return x_seq_out


    def backward(self, x_seq_out_grad: ndarray):
        '''
        Realiza el paso hacia atrás (backpropagation) para la secuencia de salida de la capa GRU.

        param x_seq_out_grad: numpy array con forma (batch_size, sequence_length, output_size), el gradiente de la pérdida.
        return x_seq_in_grad: numpy array con forma (batch_size, sequence_length, vocab_size), el gradiente de la entrada.
        '''
        
        batch_size = x_seq_out_grad.shape[0]
        
        # Inicializa los gradientes del estado oculto
        h_in_grad = np.zeros((batch_size, self.hidden_size))        
        
        num_chars = x_seq_out_grad.shape[1]
        
        # Inicializa el gradiente de la entrada de la secuencia
        x_seq_in_grad = np.zeros((batch_size, num_chars, self.vocab_size))
        
        # Itera en reversa a través de cada paso de tiempo en la secuencia
        for t in reversed(range(num_chars)):
            x_out_grad = x_seq_out_grad[:, t, :]  # Gradiente de la salida en el paso t

            # Realiza el paso hacia atrás en el nodo GRU correspondiente
            grad_out, h_in_grad = self.cells[t].backward(x_out_grad, h_in_grad, self.params)
        
            x_seq_in_grad[:, t, :] = grad_out  # Guarda el gradiente de la entrada en el paso t
        
        return x_seq_in_grad


### Ejemplos

In [ ]:
# Capa única de LSTM

capas1 = [LSTMLayer(hidden_size=256, output_size=62, weight_scale=0.01)]
mod = RNNModel(layers=capas1,
               vocab_size=62, sequence_length=25,
               loss=SoftmaxCrossEntropy())
optim = AdaGrad(lr=0.01, gradient_clipping=True)
trainer = RNNTrainer('input.txt', mod, optim, batch_size=3)
trainer.train(1000, sample_every=100)

In [ ]:
## Modelos con multiples capas

capas2 = [RNNLayer(hidden_size=256, output_size=128, weight_scale=0.1),
           LSTMLayer(hidden_size=256, output_size=62, weight_scale=0.01)]
mod = RNNModel(layers=capas2,
               vocab_size=62, sequence_length=25,
               loss=SoftmaxCrossEntropy())
optim = AdaGrad(lr=0.01, gradient_clipping=True)
trainer = RNNTrainer('input.txt', mod, optim, batch_size=32)
trainer.train(2000, sample_every=100)

In [ ]:
capas2 = [LSTMLayer(hidden_size=256, output_size=128, weight_scale=0.1),
           LSTMLayer(hidden_size=256, output_size=62, weight_scale=0.01)]
mod = RNNModel(layers=capas2,
               vocab_size=62, sequence_length=25,
               loss=SoftmaxCrossEntropy())
optim = SGD(lr=0.01, gradient_clipping=True)
trainer = RNNTrainer('input.txt', mod, optim, batch_size=32)
trainer.train(2000, sample_every=100)

In [ ]:
capas3 = [GRULayer(hidden_size=256, output_size=128, weight_scale=0.1),
           LSTMLayer(hidden_size=256, output_size=62, weight_scale=0.01)]
mod = RNNModel(layers=capas3,
               vocab_size=62, sequence_length=25,
               loss=SoftmaxCrossEntropy())
optim = AdaGrad(lr=0.01, gradient_clipping=True)
trainer = RNNTrainer('input.txt', mod, optim, batch_size=32)
trainer.train(2000, sample_every=100)

Los ejemplos anteriores configuran y entrenan un modelo de red neuronal recurrente que opera a nivel de carácter, utilizando diferentes combinaciones de capas RNN, LSTM y GRU. La salida generada indica que el modelo está generando texto basado en secuencias de caracteres, replicando en parte el estilo del texto de entrada de "King Lear" de Shakespeare.

#### **Ejercicio 1: Comprensión de la arquitectura del modelo**

**Objetivo:** Entender la estructura del modelo configurado en el código y cómo se interrelacionan las diferentes capas.

**Instrucciones:**

1. **Identificación de capas:**
   - Observa las diferentes configuraciones de capas (`capas2`, `capas3`) en el código proporcionado.
   - Describe la arquitectura de cada configuración, especificando el tipo de capas utilizadas y sus parámetros (`hidden_size`, `output_size`, `weight_scale`).

2. **Flujo de datos:**
   - Explica cómo fluye la información desde la entrada hasta la salida en cada una de las configuraciones.
   - ¿Cómo interactúan las capas entre sí durante el paso hacia adelante?

**Puntos clave a considerar:**

- Diferencias entre `RNNLayer`, `LSTMLayer` y `GRULayer`.
- Cómo se combinan múltiples capas para formar una red más profunda.
- La importancia de `output_size` en la última capa para la generación de salidas.


#### **Ejercicio 2: Nivel de análisis del modelo**

**Objetivo:** Determinar si el modelo opera a nivel de **carácter** o de **palabra** y comprender las implicaciones de esta elección.

**Instrucciones:**

1. **Análisis del tamaño del vocabulario:**
   - Basándote en `vocab_size=62`, ¿a qué nivel está operando el modelo? Justifica tu respuesta.

2. **Interpretación de la salida generada:**
   - Observa la salida proporcionada:
     ```
     he lots I kyour bjt?n, uld ofpyo: lruysAfoV havehe,
     ve and , Vminne, vay, ' -your Lralns Stry, Bear of's, I bvall.
     I cew,ew, vadd
     Le my Vruny:
     I O spraiJ, bu larrtwes a ' hat! I be the !ue
     co a parlde y el texto de input.txt tiene la forma de: That, poor contempt, or claim'd thou slept so faithful,
     ...
     ```
     - ¿Confirma esta salida tu conclusión sobre el nivel de análisis? Explica por qué.

3. **Ventajas y desventajas:**
   - Discute las ventajas y desventajas de entrenar un modelo a nivel de carácter frente a a nivel de palabra.

**Puntos clave a considerar:**

- Cómo el tamaño del vocabulario influye en el nivel de análisis.
- Coherencia semántica vs. creatividad en la generación de texto.
- Complejidad computacional y requisitos de memoria.


#### **Ejercicio 3: Modificación de la arquitectura**

**Objetivo:** Modificar la arquitectura del modelo para experimentar con diferentes configuraciones y observar sus efectos.

**Instrucciones:**

1. **Añadir una capa adicional:**
   - Modifica la segunda configuración de `capas2` para incluir una tercera capa `GRULayer` con `hidden_size=128` y `output_size=256`.
   - Actualiza la creación del modelo `RNNModel` para incluir esta nueva capa.

2. **Cambiar el tamaño de la secuencia:**
   - Ajusta `sequence_length` de 25 a 50 en el modelo modificado.
   - Observa y describe cómo este cambio podría afectar el entrenamiento y la generación de texto.

3. **Implementar una capa de dropout:**
   - Introduce una capa de Dropout entre las capas GRU y LSTM para reducir el sobreajuste.
   - Explica cómo y por qué el Dropout puede ayudar en este contexto.

**Puntos clave a considerar:**

- Impacto de agregar más capas en la capacidad del modelo para aprender patrones complejos.
- Cómo el tamaño de la secuencia afecta la captura de dependencias a largo plazo.
- Beneficios de la regularización mediante Dropout.

#### **Ejercicio 4: Implementación de una nueva función de activación**

**Objetivo:** Experimentar con diferentes funciones de activación para potencialmente mejorar el rendimiento del modelo.

**Instrucciones:**

1. **Seleccionar una función de activación alternativa:**
   - Elige una función de activación diferente a la sigmoide y tangente hiperbólica (por ejemplo, **ReLU** o **Leaky ReLU**).

2. **Modificar el código de las capas:**
   - Implementa la función de activación seleccionada en las puertas y candidatos de estado oculto de las clases `LSTMNode` y `GRUNode`.
   - Asegúrate de actualizar también las derivadas correspondientes para el paso hacia atrás.

3. **Entrenar el modelo modificado:**
   - Entrena el modelo con la nueva función de activación utilizando el mismo conjunto de datos y parámetros de entrenamiento.
   - Compara el rendimiento (pérdida y calidad del texto generado) con la configuración original.

**Puntos clave a considerar:**

- Cómo diferentes funciones de activación afectan la capacidad del modelo para aprender.
- Posibles mejoras en la velocidad de convergencia y en la capacidad de manejar gradientes.


#### **Ejercicio 5: Evaluación del impacto del optimizer**

**Objetivo:** Analizar cómo diferentes algoritmos de optimización afectan el entrenamiento del modelo.

**Instrucciones:**

1. **Comparar AdaGrad y SGD:**
   - Utiliza ambas configuraciones de capas (`capas2` con `AdaGrad` y `capas2` con `SGD`) para entrenar el modelo en el mismo conjunto de datos.

2. **Registrar métricas de entrenamiento:**
   - Durante el entrenamiento, registra la pérdida y cualquier otra métrica relevante para ambas configuraciones.

3. **Analizar los resultados:**
   - Compara la convergencia de la pérdida entre AdaGrad y SGD.
   - Discute cuál optimizador muestra un mejor rendimiento y por qué podría ser así en este contexto.

**Puntos clave a considerar:**

- Características de AdaGrad y SGD que afectan su rendimiento.
- Cómo la tasa de aprendizaje y el clipping de gradientes interactúan con cada optimizador.
- Escenarios en los que un optimizador puede ser preferible sobre otro.


#### **Ejercicio 6: Implementación de regularización por dropout**

**Objetivo:** Reducir el sobreajuste mediante la implementación de Dropout en la arquitectura del modelo.

**Instrucciones:**

1. **Añadir capas de dropout:**
   - Inserta capas de Dropout con una tasa de 0.5 después de cada capa recurrente (RNN, LSTM, GRU) en la configuración `capas3`.

2. **Modificar el flujo de datos:**
   - Asegúrate de que las capas de Dropout se apliquen correctamente durante el paso hacia adelante y hacia atrás.

3. **Entrenar y evaluar:**
   - Entrena el modelo modificado en el mismo conjunto de datos.
   - Compara la pérdida de entrenamiento y validación con la configuración sin Dropout.
   - Evalúa si la inclusión de Dropout mejora la generalización del modelo.

**Puntos clave a considerar:**

- Cómo Dropout ayuda a prevenir el sobreajuste al introducir ruido durante el entrenamiento.
- Impacto de Dropout en la velocidad de entrenamiento y en la capacidad del modelo para aprender patrones relevantes.


#### **Ejercicio 7: Depuración de errores en la implementación del backward**

**Objetivo:** Revisar y corregir posibles errores en la implementación del método `backward` de la clase `GRUNode`.

**Instrucciones:**

1. **Revisar las derivadas:**
   - Analiza el método `backward` de la clase `GRUNode` y verifica si las derivadas calculadas corresponden correctamente a las ecuaciones matemáticas de una GRU.

2. **Identificar inconsistencias:**
   - Busca posibles inconsistencias o errores en la propagación de gradientes, especialmente en las puertas de reinicio y actualización.

3. **Corregir el código:**
   - Realiza las modificaciones necesarias para asegurar que las derivadas se calculan correctamente.
   - Asegúrate de que las actualizaciones de los parámetros reflejan las derivadas correctas.

4. **Validar las correcciones:**
   - Entrena el modelo después de las correcciones y verifica si la pérdida disminuye de manera consistente.
   - Genera muestras de texto para evaluar si la calidad ha mejorado.

**Puntos clave a considerar:**

- Importancia de las derivadas correctas para el entrenamiento efectivo del modelo.
- Cómo los errores en la retropropagación pueden afectar la convergencia y la calidad de las predicciones.


#### **Ejercicio 8: Experimentación con el tamaño del lote (Batch Size)**

**Objetivo:** Evaluar cómo diferentes tamaños de lote afectan el entrenamiento y el rendimiento del modelo.

**Instrucciones:**

1. **Probar diferentes tamaños de lote:**
   - Reentrena el modelo utilizando diferentes tamaños de lote, por ejemplo, 16, 32, 64 y 128.

2. **Registrar el rendimiento:**
   - Para cada tamaño de lote, registra la pérdida de entrenamiento y la calidad de las muestras generadas.

3. **Analizar los resultados:**
   - Compara cómo el tamaño del lote afecta la velocidad de convergencia y la estabilidad del entrenamiento.
   - Discute las ventajas y desventajas de usar tamaños de lote más pequeños vs. más grandes.

**Puntos clave a considerar:**

- Influencia del tamaño del lote en la estimación del gradiente.
- Compromiso entre velocidad de entrenamiento y calidad de las actualizaciones de parámetros.
- Impacto en el uso de memoria y en la capacidad de paralelización.



#### **Ejercicio 9: Implementación de gradient clipping**

**Objetivo:** Asegurar la estabilidad del entrenamiento mediante la implementación y ajuste del **gradient clipping**.

**Instrucciones:**

1. **Entender gradient clipping:**
   - Investiga qué es el gradient clipping y por qué es útil en el entrenamiento de redes recurrentes.

2. **Implementar gradient clipping:**
   - Si no está ya implementado, añade una función de gradient clipping en el optimizador `AdaGrad` y `SGD`.
   - Define un umbral adecuado para el clipping, por ejemplo, 5.0.

3. **Ajustar el umbral:**
   - Experimenta con diferentes valores de umbral para el clipping y observa cómo afectan al entrenamiento.

4. **Evaluar la efectividad:**
   - Compara la estabilidad y la convergencia del entrenamiento con y sin gradient clipping.
   - Observa si el modelo evita problemas como gradientes explosivos.

**Puntos clave a considerar:**

- Cómo gradient clipping ayuda a prevenir gradientes excesivamente grandes que pueden desestabilizar el entrenamiento.
- Selección adecuada del umbral para balancear la preservación de información y la estabilidad.


#### **Ejercicio 10: Generación de texto con el modelo entrenado**

**Objetivo:** Utilizar el modelo entrenado para generar texto y evaluar su capacidad para imitar el estilo del texto de entrada.

**Instrucciones:**

1. **Preparar una semilla:**
   - Selecciona una secuencia de caracteres inicial (semilla) del texto de entrada para comenzar la generación.

2. **Generar texto:**
   - Utiliza el modelo entrenado para predecir el siguiente carácter de manera iterativa, extendiendo la secuencia generada.
   - Genera al menos 500 caracteres de texto continuo.

3. **Evaluar la calidad:**
   - Analiza la coherencia, la creatividad y la fidelidad al estilo del texto de entrada.
   - Identifica patrones repetitivos o incoherencias.

4. **Experimentar con la temperatura:**
   - Implementa una función de **temperatura** que ajuste la aleatoriedad en la selección del siguiente carácter.
   - Genera texto con diferentes valores de temperatura (por ejemplo, 0.5, 1.0, 1.5) y compara los resultados.

**Puntos clave a considerar:**

- Cómo la semilla inicial influye en la generación de texto.
- Impacto de la temperatura en la diversidad y creatividad del texto generado.
- Evaluación subjetiva de la calidad del texto en términos de coherencia y estilo.


#### **Ejercicio 11: Implementación de early stopping**

**Objetivo:** Mejorar la eficiencia del entrenamiento y prevenir el sobreajuste mediante la implementación de **early stopping**.

**Instrucciones:**

1. **Definir una métrica de monitoreo:**
   - Selecciona una métrica para monitorear durante el entrenamiento, como la pérdida de validación.

2. **Implementar early stopping:**
   - Modifica la clase `RNNTrainer` para que detenga el entrenamiento si la métrica seleccionada no mejora después de un número determinado de iteraciones (por ejemplo, 10).

3. **Dividir el conjunto de datos:**
   - Separa una porción del `input.txt` como conjunto de validación.

4. **Entrenar con early stopping:**
   - Entrena el modelo utilizando early stopping y observa si el entrenamiento se detiene antes de alcanzar las 2000 iteraciones.

5. **Comparar resultados:**
   - Compara el rendimiento del modelo con y sin early stopping en términos de pérdida y calidad de generación de texto.

**Puntos clave a considerar:**

- Cómo early stopping ayuda a prevenir el sobreajuste al detener el entrenamiento cuando el rendimiento en datos de validación ya no mejora.
- Selección adecuada del número de iteraciones de paciencia antes de detener el entrenamiento.
- Impacto en el tiempo de entrenamiento y en la generalización del modelo.

#### **Ejercicio 12: Adaptación del modelo para datos a nivel de palabra**

**Objetivo:** Modificar el modelo para operar a nivel de palabra en lugar de a nivel de carácter.

**Instrucciones:**

1. **Aumentar el tamaño del vocabulario:**
   - Procesa el archivo `input.txt` para tokenizar el texto en palabras.
   - Crea un mapeo de palabras a índices y ajusta `vocab_size` en el modelo para reflejar el número total de palabras únicas.

2. **Modificar la entrada y salida:**
   - Cambia la representación de la entrada y salida del modelo para que cada unidad corresponda a una palabra en lugar de a un carácter.
   - Asegúrate de que las capas de salida tengan un tamaño igual al nuevo `vocab_size`.

3. **Actualizar el modelo:**
   - Ajusta las capas del modelo para manejar el nuevo tamaño del vocabulario.
   - Considera reducir el `hidden_size` si el modelo se vuelve demasiado grande.

4. **Entrenar el modelo:**
   - Entrena el modelo adaptado en el nuevo conjunto de datos a nivel de palabra.
   - Genera muestras de texto y evalúa la coherencia y la calidad en comparación con el modelo a nivel de carácter.

5. **Comparar rendimiento:**
   - Discute las diferencias en la generación de texto y el rendimiento del modelo al operar a nivel de palabra frente a carácter.

**Puntos clave a considerar:**

- Cómo la tokenización afecta la representación de los datos.
- Impacto del aumento del tamaño del vocabulario en la complejidad computacional y en la capacidad del modelo.
- Ventajas de trabajar a nivel de palabra en términos de coherencia semántica.


#### **Ejercicio 13: Visualización de las activaciones de las capas**

**Objetivo:** Comprender cómo las diferentes capas del modelo responden a ciertas entradas mediante la visualización de sus activaciones.

**Instrucciones:**

1. **Seleccionar una entrada:**
   - Elige una secuencia de caracteres específica del conjunto de datos de entrenamiento.

2. **Capturar activaciones:**
   - Modifica las clases de las capas (`RNNLayer`, `LSTMLayer`, `GRULayer`) para almacenar las activaciones (salidas intermedias) durante el paso hacia adelante.

3. **Visualizar las activaciones:**
   - Utiliza herramientas de visualización como **matplotlib** para crear gráficos de las activaciones de las diferentes capas.
   - Observa cómo las activaciones varían a lo largo de la secuencia de entrada.

4. **Interpretar los resultados:**
   - Analiza qué patrones o comportamientos se pueden observar en las activaciones.
   - Discute cómo las diferentes capas contribuyen al procesamiento de la información.

**Puntos clave a considerar:**

- Cómo las activaciones de las capas recurrentes reflejan la memoria y la atención del modelo.
- Identificación de patrones recurrentes o estados de activación que corresponden a ciertos caracteres o contextos.


#### **Ejercicio 14: Implementación de batch normalization**

**Objetivo:** Mejorar la estabilidad y velocidad de entrenamiento mediante la implementación de **Batch Normalization** en el modelo.

**Instrucciones:**

1. **Investigar batch normalization:**
   - Comprende qué es Batch Normalization y cómo puede aplicarse en redes recurrentes.

2. **Añadir batch normalization:**
   - Introduce capas de Batch Normalization después de cada capa recurrente (RNN, LSTM, GRU) en la arquitectura del modelo.
   - Implementa las funciones necesarias para normalizar las activaciones durante el paso hacia adelante y ajustar los parámetros durante el entrenamiento.

3. **Entrenar el modelo con batch normalization:**
   - Entrena el modelo modificado y observa los efectos en la convergencia y en la calidad de las muestras generadas.

4. **Comparar con la configuración original:**
   - Compara el rendimiento del modelo con Batch Normalization frente al modelo sin ella.
   - Discute los beneficios y posibles inconvenientes observados.

**Puntos clave a considerar:**

- Cómo Batch Normalization ayuda a estabilizar las distribuciones de activación y acelera el entrenamiento.
- Consideraciones especiales al aplicar Batch Normalization en arquitecturas recurrentes.


#### **Ejercicio 15: Guardar y cargar el modelo entrenado**

**Objetivo:** Implementar mecanismos para guardar el estado del modelo durante el entrenamiento y cargarlo posteriormente para la generación de texto.

**Instrucciones:**

1. **Implementar funciones de guardado:**
   - Añade métodos en la clase `RNNModel` para guardar los pesos y sesgos de todas las capas en un archivo (por ejemplo, en formato JSON o pickle).

2. **Implementar funciones de carga:**
   - Añade métodos para cargar los pesos y sesgos desde un archivo guardado y restaurar el estado del modelo.

3. **Entrenar y guardar el modelo:**
   - Durante el entrenamiento, guarda el estado del modelo cada cierto número de iteraciones (por ejemplo, cada 500 iteraciones).

4. **Cargar el modelo y generar texto:**
   - Detén el entrenamiento y carga el modelo desde un archivo guardado.
   - Genera texto utilizando el modelo cargado y verifica que las salidas sean consistentes con el estado guardado.

**Puntos clave a considerar:**

- Importancia de guardar el estado del modelo para reproducibilidad y para evitar entrenamientos repetidos.
- Manejo adecuado de formatos de archivo para preservar la integridad de los pesos y sesgos.
- Verificación de la consistencia entre el modelo antes y después de la carga.


#### **Ejercicio 16: Optimización del rendimiento del modelo**

**Objetivo:** Mejorar el rendimiento del modelo mediante la optimización de hiperparámetros y la arquitectura.

**Instrucciones:**

1. **Ajustar hiperparámetros:**
   - Experimenta con diferentes valores de `hidden_size`, `output_size`, y `weight_scale`.
   - Observa cómo estos cambios afectan la capacidad del modelo para aprender y generar texto.

2. **Implementar técnicas de regularización:**
   - Además de Dropout, prueba otras técnicas como **L2 regularization** o **early stopping** (si no lo has hecho en ejercicios anteriores).

3. **Reducir el tiempo de entrenamiento:**
   - Optimiza el código para mejorar la eficiencia computacional, por ejemplo, utilizando operaciones vectorizadas de NumPy de manera más efectiva.
   - Considera implementar técnicas como **mini-batch training** si aún no están implementadas.

4. **Evaluar mejoras:**
   - Compara el rendimiento del modelo antes y después de las optimizaciones realizadas.
   - Discute qué cambios fueron más efectivos y por qué.

**Puntos clave a considerar:**

- Balance entre la complejidad del modelo y la capacidad de generalización.
- Impacto de la regularización en la prevención del sobreajuste.
- Estrategias para acelerar el entrenamiento sin sacrificar la calidad del modelo.

#### **Ejercicio 17: Implementación de una función de activación personalizada**

**Objetivo:** Crear e integrar una función de activación personalizada en el modelo para explorar su impacto en el aprendizaje.

**Instrucciones:**

1. **Diseñar una función de activación:**
   - Crea una nueva función de activación, por ejemplo, una variante de ReLU como **Leaky ReLU**:
     $$
     f(x) = 
     \begin{cases} 
     x & \text{si } x > 0 \\
     \alpha x & \text{si } x \leq 0 
     \end{cases}
     $$
     donde $\alpha$ es un pequeño valor, como 0.01.

2. **Implementar la función y su derivada:**
   - Escribe las funciones en Python para la activación y su derivada.

3. **Integrar la función en las capas recurrentes:**
   - Modifica las clases `LSTMNode` y `GRUNode` para utilizar la nueva función de activación en lugar de las funciones estándar.

4. **Entrenar el modelo con la activación personalizada:**
   - Entrena el modelo y evalúa cómo afecta la nueva función de activación al rendimiento y a la calidad del texto generado.

5. **Comparar con la función de activación original:**
   - Discute las diferencias observadas en el comportamiento del modelo con y sin la función de activación personalizada.

**Puntos clave a considerar:**

- Cómo las diferentes funciones de activación afectan la capacidad del modelo para aprender y manejar el flujo de gradientes.
- Beneficios de funciones de activación que permiten pequeñas pendientes en regiones negativas para prevenir gradientes muertos.

#### **Ejercicio 18: Evaluación del modelo con datos de prueba**

**Objetivo:** Evaluar el desempeño del modelo utilizando un conjunto de datos de prueba independiente.

**Instrucciones:**

1. **Dividir el conjunto de datos:**
   - Separa una porción del archivo `input.txt` como conjunto de prueba, asegurándote de que no se use durante el entrenamiento.

2. **Modificar la clase `RNNTrainer`:**
   - Añade funcionalidad para evaluar el modelo en el conjunto de prueba después de cada cierto número de iteraciones de entrenamiento.

3. **Calcular métricas de evaluación:**
   - Implementa métricas adicionales como **Perplexity** para evaluar la calidad del modelo en datos no vistos.

4. **Analizar el rendimiento:**
   - Compara las métricas obtenidas en el conjunto de entrenamiento y en el de prueba.
   - Discute posibles indicaciones de sobreajuste o subajuste.

**Puntos clave a considerar:**

- Importancia de un conjunto de prueba para evaluar la capacidad de generalización del modelo.
- Interpretación de métricas como Perplexity en el contexto de modelos de lenguaje.


#### **Ejercicio 19: Implementación de un pipeline de preprocesamiento mejorado**

**Objetivo:** Mejorar el preprocesamiento de los datos para optimizar el entrenamiento del modelo.

**Instrucciones:**

1. **Normalización de datos:**
   - Implementa técnicas de normalización para las entradas, como **one-hot encoding** para los caracteres o palabras.

2. **Gestión de datos desbalanceados:**
   - Si ciertas clases (caracteres o palabras) son significativamente más frecuentes, implementa técnicas para balancear el conjunto de datos, como **undersampling** o **oversampling**.

3. **Implementar embeddings:**
   - Añade una capa de embeddings que transforma las representaciones one-hot en vectores densos de menor dimensión.
   - Ajusta la arquitectura del modelo para integrar esta capa.

4. **Entrenar el modelo con el pipeline mejorado:**
   - Observa cómo estas mejoras afectan la capacidad del modelo para aprender patrones y generar texto coherente.

**Puntos clave a considerar:**

- Cómo la representación de los datos afecta la eficiencia y efectividad del aprendizaje del modelo.
- Beneficios de los embeddings en la captura de relaciones semánticas entre palabras o caracteres.


#### **Ejercicio 20: Comparación entre RNN, LSTM y GRU**

**Objetivo:** Evaluar y comparar el rendimiento de diferentes arquitecturas recurrentes en la tarea de generación de texto.

**Instrucciones:**

1. **Configurar modelos separados:**
   - Define tres configuraciones de modelos:
     - **Modelo A:** Solo con capas `RNNLayer`.
     - **Modelo B:** Solo con capas `LSTMLayer`.
     - **Modelo C:** Solo con capas `GRULayer`.

2. **Entrenar cada modelo:**
   - Entrena cada modelo utilizando el mismo conjunto de datos, optimizador y parámetros de entrenamiento.

3. **Generar y comparar salidas:**
   - Genera muestras de texto con cada modelo.
   - Compara la coherencia, creatividad y fidelidad al estilo del texto de entrada.

4. **Analizar métricas de rendimiento:**
   - Evalúa y compara métricas como la pérdida de entrenamiento, velocidad de convergencia y estabilidad durante el entrenamiento.

5. **Discusión:**
   - Discute cuál de las arquitecturas mostró un mejor rendimiento y por qué podría ser así.
   - Considera aspectos como la capacidad para capturar dependencias a largo plazo y la eficiencia computacional.

**Puntos clave a considerar:**

- Fortalezas y debilidades de RNN, LSTM y GRU en tareas de modelado secuencial.
- Cómo las diferencias en las puertas y el manejo de la memoria afectan el aprendizaje y la generación de texto.


#### **Ejercicio 21: Implementación de técnicas de data augmentation**

**Objetivo:** Aumentar la diversidad del conjunto de datos de entrenamiento para mejorar la generalización del modelo.

**Instrucciones:**

1. **Definir Técnicas de data augmentation:**
   - Implementa técnicas como **sinónimos de sustitución** (para modelos a nivel de palabra) o **inserción de ruido** (para modelos a nivel de carácter).

2. **Aplicar data Augmentation al conjunto de datos:**
   - Modifica el archivo `input.txt` aplicando las técnicas definidas para crear un conjunto de datos ampliado.

3. **Entrenar el modelo con datos aumentados:**
   - Reentrena el modelo utilizando el nuevo conjunto de datos aumentado.
   - Observa si hay mejoras en la capacidad del modelo para generar texto variado y coherente.

4. **Evaluar el impacto:**
   - Compara el rendimiento del modelo con y sin data augmentation en términos de pérdida y calidad de la generación de texto.

**Puntos clave a considerar:**

- Cómo el data augmentation puede ayudar a prevenir el sobreajuste y mejorar la robustez del modelo.
- Selección adecuada de técnicas de augmentation que mantengan la coherencia del texto.


#### **Ejercicio 22: Implementación de un mecanismo de atención**

**Objetivo:** Añadir un mecanismo de atención al modelo para mejorar la capacidad de enfocarse en partes relevantes de la secuencia de entrada.

**Instrucciones:**

1. **Investigar mecanismos de atención:**
   - Comprende los conceptos básicos de los mecanismos de atención en redes neuronales recurrentes.

2. **Diseñar e implementar la atención:**
   - Añade una capa de atención después de las capas recurrentes (`RNNLayer`, `LSTMLayer`, `GRULayer`).
   - Implementa las operaciones de cálculo de pesos de atención y combinación de contextos.

3. **Modificar el flujo de datos:**
   - Asegúrate de que las salidas de las capas recurrentes se combinan adecuadamente con la atención para producir la salida final.

4. **Entrenar el modelo con atención:**
   - Entrena el modelo y evalúa si la atención mejora la calidad de la generación de texto.

5. **Visualizar los pesos de atención:**
   - Implementa visualizaciones de los pesos de atención para entender en qué partes de la secuencia el modelo está enfocando su atención durante la generación.

**Puntos clave a considerar:**

- Cómo la atención permite que el modelo se enfoque dinámicamente en diferentes partes de la secuencia de entrada.
- Beneficios de la atención en la captura de dependencias a largo plazo y en la generación de salidas más coherentes.


### **Ejercicio 23: Implementación de bidirectional RNN**

**Objetivo:** Mejorar la capacidad del modelo para capturar información contextual desde ambas direcciones de la secuencia.

**Instrucciones:**

1. **Comprender bidirectional RNN:**
   - Investiga cómo funcionan las RNN bidireccionales y sus ventajas.

2. **Modificar la arquitectura del modelo:**
   - Cambia las capas recurrentes (`RNNLayer`, `LSTMLayer`, `GRULayer`) para que sean bidireccionales.
   - Esto implica crear dos instancias de cada capa: una que procesa la secuencia en orden normal y otra en orden inverso.

3. **Concatenar las salidas:**
   - Combina las salidas de las direcciones forward y backward antes de pasar a la siguiente capa o a la capa de salida.

4. **Entrenar el modelo bidireccional:**
   - Entrena el modelo modificado y compara su rendimiento con el modelo unidireccional.

5. **Evaluar mejoras:**
   - Analiza si la capacidad de capturar contexto bidireccional mejora la coherencia y relevancia del texto generado.

**Puntos clave a considerar:**

- Cómo las RNN bidireccionales pueden capturar mejor el contexto global de la secuencia.
- Impacto en la complejidad computacional y en el tiempo de entrenamiento.


#### **Ejercicio 24: Implementación de early fusion de características**

**Objetivo:** Integrar características adicionales al modelo para enriquecer la representación de la entrada.

**Instrucciones:**

1. **Definir características adicionales:**
   - Selecciona características que puedan enriquecer la representación de los caracteres o palabras, como **posiciones en la secuencia**, **características gramaticales** (para modelos a nivel de palabra), etc.

2. **Modificar el preprocesamiento de datos:**
   - Extrae y agrega estas características adicionales a la representación de la entrada.

3. **Actualizar las capas de entrada:**
   - Ajusta las capas recurrentes para aceptar las características adicionales, aumentando el tamaño de la entrada si es necesario.

4. **Entrenar y evaluar el modelo:**
   - Entrena el modelo con las nuevas características y evalúa si mejoran la capacidad del modelo para generar texto coherente y estilísticamente consistente.

**Puntos clave a considerar:**

- Cómo las características adicionales pueden aportar información relevante que no está presente en la secuencia de caracteres o palabras por sí sola.
- Manejo adecuado de la dimensionalidad aumentada en las capas de entrada.


#### **Ejercicio 25: Implementación de regularización por DropConnect**

**Objetivo:** Introducir una variante de regularización conocida como **DropConnect** en lugar de Dropout para explorar sus efectos en el modelo.

**Instrucciones:**

1. **Entender DropConnect:**
   - Investiga qué es DropConnect y cómo difiere de Dropout.

2. **Implementar DropConnect:**
   - Añade DropConnect a las capas de conexión (pesos) de las capas recurrentes (`RNNLayer`, `LSTMLayer`, `GRULayer`).
   - Define una tasa de conexión que determine la probabilidad de mantener una conexión activa.

3. **Integrar en el flujo de datos:**
   - Asegúrate de que DropConnect se aplica durante el entrenamiento pero no durante la inferencia.

4. **Entrenar el modelo con DropConnect:**
   - Entrena el modelo y observa cómo afecta la regularización al rendimiento y a la generación de texto.

5. **Comparar con dropout:**
   - Compara el impacto de DropConnect frente a Dropout en términos de pérdida de entrenamiento y calidad del texto generado.

**Puntos clave a considerar:**

- Cómo DropConnect puede proporcionar una regularización más robusta al aplicar ruido directamente a las conexiones de pesos.
- Impacto en la capacidad del modelo para generalizar y en la estabilidad del entrenamiento.

In [ ]:
## Tus respuestas